In [1]:
import os
import tarfile

if not os.path.isdir('aclImdb'):

    with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
        tar.extractall()

In [4]:
import pyprind
import pandas as pd
import os

basepath = 'aclImdb'
labels = {'pos': 1, 'neg': 0}

pbar = pyprind.ProgBar(50000)
data = []

for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file),
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            data.append([txt, labels[l]])
            pbar.update()

df = pd.DataFrame(data, columns=['review', 'sentiment'])


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


In [7]:
import numpy as np
import pandas as pd

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

df.to_csv('movie_data.csv', index=False, encoding='utf-8')

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,at a Saturday matinee in my home town. I went ...,0
1,I love this movie. It is the first film Master...,1
2,"In the voice over which begins the film, Hughi...",1


In [17]:
### 본격적인 BoW 이해해보기

from sklearn.feature_extraction.text import CountVectorizer

# 사이킷런에는 역시 BoW를 바로 사용해볼 수 있는 모델이 존재함.
count = CountVectorizer()
docs = np.array(['The sun is shining',
                'The weather is sweet',
                'The sun is shining, the weather is sweet and one and one is two'])
# 임시 문장을 만들어주고 count의 fit_transform 메서드를 통해서 단어 사전을 만들 수 있음.
bag = count.fit_transform(docs)
print(count.vocabulary_)

# 여기서 딕셔너리의 value는 단어의 고유한 인덱스임.
# 밑 셀의 예시를 보면 쉽게 이해할 수 있음.

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [ ]:
print(bag.toarray())

# 예를 들어 1번째 문장을 보면, 1번 인덱스인 is가 1번 포함됐으니 1로 되어있고, 4번 인덱스인 sun이 포함되어있으니 1로 되어있고
# 이런식으로 인덱스별로 매칭되어있음.
# 3번째 문장은 is가 총 3번 포함되어 있으니 1번 인덱스가 3으로 되어있는 모습.


### 참고로 이번에 단어 사전을 만들때 1단어 단위(1-gram)로 끊어서 그렇지 n-gram 단위로 끊어서 단어사전을 만들 수 있음. 

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [ ]:
### TF-IDF로 특성 벡터에서 단어의 가중치를 낮춰보기

from sklearn.feature_extraction.text import TfidfTransformer

# L2 norm을 사용하여 정규화
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

# tf-idf를 사용하면 is 같은 문장의 결정적인 요소가 아니지만 쓸데없이 높게 빈도수가 잡히는걸 줄여줄 수 있음.

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [ ]:
### 텍스트 데이터 정제
import re

df.loc[0, 'review'][-50:]

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

preprocessor(df.loc[0, 'review'][-50:])

df['review'] = df['review'].apply(preprocessor)
df['review'].map(preprocessor)

# 데이터셋이 HTML에서 가져온 문장이 있다보니 tag가 있는 리뷰가 있어서 그걸 없애기 위해서
# 전처리 함수를 통해 dataframe의 모든 문장을 전처리
# 정규 표현식인 Regular Expression(re) 라이브러리를 가져와서 쉽게 전처리 할 수 있음.

0        at a saturday matinee in my home town i went w...
1        i love this movie it is the first film master ...
2        in the voice over which begins the film hughie...
3         spoiler alert the point is though that i didn...
4        this is an excellent film no it s not mel gibs...
                               ...                        
49995    although the director tried the filming was ma...
49996    it has been about 50 years since a movie has b...
49997     bar hopping seems to be trying to be about th...
49998    this awful effort just goes to show what happe...
49999    yes why among the filmmakers that came out in ...
Name: review, Length: 50000, dtype: object

In [ ]:
### 문서를 토큰 단위로 나누기

def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

# 정말 단순하게 1줄짜리 코드로 단어를 나눌 수 있음.

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [ ]:
### 토큰을 어간형태로 뽑아주는 어간추출 만들기

from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runners like running and thus they run')

# 이런식으로 단어의 원형으로 뽑아낼 수 있음.
# 하지만, 어간 추출은 실제로 사용하지 않는 단어들을 뽑아 낼 수 있음.

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [ ]:
### 불용어 제거

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

# 불용어는 모든 종류의 텍스트에 아주 흔하게 단어. 관사나, 동사중에서 매우 자주쓰이는 단어들 그런걸 불용어라고함.
# 그런건 사실상 감성분석을 할때 의미 없는 단어들인데 빈도수만 높게 잡히기 때문에 그냥 없애버리는 불용어 제거 기법을 사용함.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\조승현\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['runner', 'like', 'run', 'run', 'lot']

In [ ]:
# 문서분류를 위한 로지스틱 회귀

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]
